In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
import torch

In [4]:
import pyro
import deepppl

In [5]:
sns.set_style('whitegrid')


### Basic example showing the interface of *DeepPPL*
Similar to the coin model but this time using a `guide` to do stochastic variational inference


In [6]:
with open('../tests/good/coin_guide.stan', 'r') as source:
    print(source.read())

data {
  int N;
  int<lower=0,upper=1> x[N];
}
parameters {
  real<lower=0,upper=1> z;
}
model {
  z ~ beta(1, 1);
  x ~ bernoulli(z);
}

guide parameters {
 real<lower=0> alpha_q;
 real<lower=0> beta_q;
}

guide {
  z ~ beta(alpha_q, beta_q);
}


In [7]:
model = deepppl.PyroModel(model_file = '../tests/good/coin_guide.stan')

In [8]:
svi = model.svi()

Two positive and eight negative observations

In [9]:
x = torch.Tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0])

In [10]:
for step in range(10000):
    svi.step(10, x)
    if step % 100 == 0:
        print('.', end='')

....................................................................................................

In [11]:
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

We know that the posterior distribution should be a `Beta` distribution with parameters $$\alpha = 1 + 2$$ and $$\beta = 1 + 8$$

In [12]:
alpha_q, beta_q

(2.4432687759399414, 7.367493629455566)

In [13]:
alpha_q/(alpha_q + beta_q)

0.24903964391149114

To do more experiments, the store must be cleared

In [14]:
pyro.clear_param_store()